[GitHub: NVIDIA-AI-IOT/deepstream_python_apps](https://github.com/NVIDIA-AI-IOT/deepstream_python_apps)

# Connect to IMX708 CSI camera

[GitHub: JetsonHacksNano/CSI-Camera](https://github.com/JetsonHacksNano/CSI-Camera)  

How To Check Camera Connection  
```ls /dev/video*```  


Install Raspberry Pi Camera Module v3 IMX708 Driver.  
[RidgeRun Developer Wiki](https://developer.ridgerun.com/wiki/index.php/Raspberry_Pi_Camera_Module_3_IMX708_Linux_driver_for_Jetson)
```
View the Pixel format
```
v4l2-ctl --list-formats-ext  
v4l2-ctl --list-devices
v4l2-ctl --device /dev/video0 --list-formats-ext
v4l2-ctl --device /dev/video0 --list-ctrls
```
Preview without converting
```
SENSOR_ID=0 
FRAMERATE=14 
gst-launch-1.0 nvarguscamerasrc sensor-id=$SENSOR_ID ! "video/x-raw(memory:NVMM),width=4608,height=2592,framerate=$FRAMERATE/1" ! queue ! nvegltransform ! nveglglessink
```
Preview with converting
```
sensor_id=0
capture_width=4608
capture_height=2592
display_width=960
display_height=540
framerate=14
flip_method=0
gst-launch-1.0 nvarguscamerasrc sensor-id=$sensor_id ! "video/x-raw(memory:NVMM), width=$capture_width, height=$capture_height, framerate=$framerate/1" ! nvvidconv flip-method=$flip_method ! "video/x-raw, width=$display_width, height=$display_height, format=(string)BGRx" ! queue ! videoconvert ! "video/x-raw, format=(string)BGR" ! nveglglessink
```

In [8]:
# MIT License
# Copyright (c) 2019-2022 JetsonHacks

# Using a CSI camera (such as the Raspberry Pi Version 2) connected to a
# NVIDIA Jetson Nano Developer Kit using OpenCV
# Drivers for the camera and OpenCV are included in the base image

import cv2

""" 
gstreamer_pipeline returns a GStreamer pipeline for capturing from the CSI camera
Flip the image by setting the flip_method (most common values: 0 and 2)
display_width and display_height determine the size of each camera pane in the window on the screen
Default 1920x1080 displayd in a 1/4 size window
"""

def gstreamer_pipeline(
    sensor_id=0,
    capture_width=4608,
    capture_height=2592,
    display_width=960,
    display_height=540,
    framerate=14,
    flip_method=0,
):
    return (
        'nvarguscamerasrc sensor-id=%d ! '
        '"video/x-raw(memory:NVMM), width=(int)%d, height=(int)%d, framerate=(fraction)%d/1" ! '
        'nvvidconv flip-method=%d ! '
        '"video/x-raw, width=(int)%d, height=(int)%d,, format=(string)BGRx" ! '
        'queue ! videoconvert ! "video/x-raw, format=(string)BGR" ! nveglglessink'
        % (
            sensor_id,
            capture_width,
            capture_height,
            framerate,
            flip_method,
            display_width,
            display_height,
        )
    )


def show_camera():
    window_title = "CSI Camera"

    # To flip the image, modify the flip_method parameter (0 and 2 are the most common)
    print(gstreamer_pipeline(flip_method=0))
    video_capture = cv2.VideoCapture(gstreamer_pipeline(flip_method=0), cv2.CAP_GSTREAMER)
    if video_capture.isOpened():
        try:
            window_handle = cv2.namedWindow(window_title, cv2.WINDOW_AUTOSIZE)
            while True:
                ret_val, frame = video_capture.read()
                # Check to see if the user closed the window
                # Under GTK+ (Jetson Default), WND_PROP_VISIBLE does not work correctly. Under Qt it does
                # GTK - Substitute WND_PROP_AUTOSIZE to detect if window has been closed by user
                if cv2.getWindowProperty(window_title, cv2.WND_PROP_AUTOSIZE) >= 0:
                    cv2.imshow(window_title, frame)
                else:
                    break 
                keyCode = cv2.waitKey(10) & 0xFF
                # Stop the program on the ESC key or 'q'
                if keyCode == 27 or keyCode == ord('q'):
                    break
        finally:
            video_capture.release()
            cv2.destroyAllWindows()
    else:
        print("Error: Unable to open camera")


if __name__ == "__main__":
    show_camera()

gst-launch-1.0 nvarguscamerasrc sensor-id=0 ! "video/x-raw(memory:NVMM), width=4608, height=2592, framerate=14/1" ! nvvidconv flip-method=0 ! "video/x-raw, width=960, height=540,, format=(string)BGRx" ! queue ! videoconvert ! "video/x-raw, format=(string)BGR" ! nveglglessink
Error: Unable to open camera


# Connect to USB camera
https://qiita.com/akira-sasaki/items/7465f6cfd6ab5977f1c9  
check connected usb camera list
```
sudo apt update
sudo apt upgrade
sudo apt install v4l-utils
v4l2-ctl --list-device 
```
show camera parameters
```
v4l2-ctl -d /dev/video0 --list-formats-ext
```
show image as YUYV 640x480 30fps
```
gst-launch-1.0 v4l2src device="/dev/video2" ! \
'video/x-raw, width=640, height=480, framerate=30/1' ! \
videoconvert ! \
nv3dsink
```

# Publish USB camera video streaming
Use MediaMTX and FFmpeg  
https://github.com/bluenviron/mediamtx